In [1]:
!pip install sidrapy
import pandas as pd
import sidrapy
import requests
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# ---
# ETAPA 1: FUNÇÃO DE COLETA DOS DADOS BRUTOS DO SIDRA/IBGE
# ---
# Esta função processa APENAS UM URL e retorna um DF padronizado.
def coletar_e_padronizar_url(url: str, padrao_coluna_regex: str) -> pd.DataFrame:
    """
    Busca dados de um único URL do SIDRA, encontra a coluna de classificação
    via regex e retorna um DataFrame com colunas padronizadas.
    """
    print(f"  Buscando dados em: {url[:60]}...")
    try:
        response = requests.get(url)
        response.raise_for_status()
        data_json = response.json()

        if len(data_json) < 2:
            print(f"    -> Aviso: URL não retornou dados: {url}")
            return pd.DataFrame() # Retorna um DataFrame vazio se não houver dados

        # Converter para DF e limpar nomes das colunas
        df_bruto = pd.DataFrame(data_json[1:])
        mapa_colunas = data_json[0]
        df_parcial = df_bruto.rename(columns=mapa_colunas)
        df_parcial.columns = df_parcial.columns.str.strip()

        # Encontrar a coluna de classificação dinamicamente
        nome_coluna_classificacao = None
        for col in df_parcial.columns:
            if re.search(padrao_coluna_regex, col, re.IGNORECASE):
                nome_coluna_classificacao = col
                break

        if not nome_coluna_classificacao:
            print(f"    -> ERRO: Nenhuma coluna para o padrão '{padrao_coluna_regex}' encontrada no URL.")
            return pd.DataFrame()

        # Selecionar e RENOMEAR PARA NOMES PADRÃO
        colunas_a_manter = ['Município (Código)', 'Ano', 'Valor', nome_coluna_classificacao]
        df_padronizado = df_parcial[colunas_a_manter].copy()
        df_padronizado.columns = ['cod_municipio', 'ano', 'valor', 'classificacao'] # Nomes padrão!

        return df_padronizado

    except Exception as e:
        print(f"    -> ERRO GERAL ao processar o URL {url}: {e}")
        return pd.DataFrame()

In [3]:
# ---
# ETAPA 2: ORQUESTRAR A COLETA USANDO A NOVA FUNÇÃO
# ---
def coletar_dados_agrupados(lista_urls: list, nome_grupo: str, padrao_regex: str) -> pd.DataFrame:
    """
    Orquestra a coleta de múltiplos URLs para um mesmo grupo (ex: água).
    """
    print(f"\n--- Iniciando coleta para o grupo: {nome_grupo} ---")

    lista_dfs_processados = [coletar_e_padronizar_url(url, padrao_regex) for url in lista_urls]

    # Concatenar os dataframes já limpos e padronizados
    df_final_grupo = pd.concat(lista_dfs_processados, ignore_index=True)

    print(f"--- Coleta para {nome_grupo} concluída. Total de {len(df_final_grupo)} linhas. ---")
    return df_final_grupo

In [4]:
# ---
# ETAPA 3: DEFINIR PARÂMETROS E EXECUTAR
# ---

# Padrões Regex para encontrar as colunas
padrao_agua = r'água.*(?<!\(Código\))$'
padrao_esgoto = r'esgotamento.*(?<!\(Código\))$'

# URLs para ÁGUA e ESGOTO
urls_agua = [
    "https://apisidra.ibge.gov.br/values/k/1748322302", # 2000 (Água)
    "https://apisidra.ibge.gov.br/values/k/-1917650165", # 2010 (Água)
    "https://apisidra.ibge.gov.br/values/k/-249458198"   # 2022 (Água)
]
urls_esgoto = [
    "https://apisidra.ibge.gov.br/values/k/-1662338465", # 2000 (Esgoto)
    "https://apisidra.ibge.gov.br/values/k/-1121183834", # 2010 (Esgoto)
    "https://apisidra.ibge.gov.br/values/k/715732111"   # 2022 (Esgoto)
]

# Executar a coleta
df_agua_raw = coletar_dados_agrupados(urls_agua, 'Pessoas com Acesso à Água', padrao_agua)
df_esgoto_raw = coletar_dados_agrupados(urls_esgoto, 'Pessoas com Acesso a Esgoto', padrao_esgoto)


--- Iniciando coleta para o grupo: Pessoas com Acesso à Água ---
  Buscando dados em: https://apisidra.ibge.gov.br/values/k/1748322302...
  Buscando dados em: https://apisidra.ibge.gov.br/values/k/-1917650165...
  Buscando dados em: https://apisidra.ibge.gov.br/values/k/-249458198...
--- Coleta para Pessoas com Acesso à Água concluída. Total de 1110 linhas. ---

--- Iniciando coleta para o grupo: Pessoas com Acesso a Esgoto ---
  Buscando dados em: https://apisidra.ibge.gov.br/values/k/-1662338465...
  Buscando dados em: https://apisidra.ibge.gov.br/values/k/-1121183834...
  Buscando dados em: https://apisidra.ibge.gov.br/values/k/715732111...
--- Coleta para Pessoas com Acesso a Esgoto concluída. Total de 2035 linhas. ---


In [5]:
# ---
# FUNÇÃO PARA CALCULAR A COBERTURA DE ÁGUA
# ---
def calcular_cobertura_agua(df_raw: pd.DataFrame) -> pd.DataFrame:
    """
    Calcula o percentual de cobertura de água de forma robusta e unificada
    para os Censos de 2000, 2010 e 2022.
    """
    print("Calculando cobertura de água com lógica unificada...")

    # Forçar a conversão da coluna 'valor' para numérico ANTES de pivotar.
    df_raw['valor'] = pd.to_numeric(df_raw['valor'], errors='coerce')
    df_raw['valor'].fillna(0, inplace=True)

    # Pivotar a tabela para que cada 'classificacao' vire uma coluna
    df_pivot = df_raw.pivot_table(
        index=['cod_municipio', 'ano'],
        columns='classificacao',
        values='valor',
        aggfunc='sum',
        fill_value=0
    ).reset_index()

    # Limpar espaços extras nos nomes das colunas para garantir a seleção
    df_pivot.columns = df_pivot.columns.str.strip()
    # --- LÓGICA DE CÁLCULO SIMPLIFICADA ---
    def calcula_percentual(row):
        # Pega o valor total diretamente, conforme sua orientação.
        total = row['Total']

        # Lógica para os Censos de 2000 e 2010
        if row['ano'] in ['2000', '2010']:
            rede_geral = row['Rede geral']
            return (rede_geral / total) * 100

        # Lógica CONTRAFACTUAL para o Censo de 2022
        elif row['ano'] == '2022':
            sem_ligacao = row['Não possui ligação com a rede geral']
            com_ligacao = total - sem_ligacao
            return (com_ligacao / total) * 100

        # Retorna Nulo para qualquer outro ano que não se encaixe nas condições
        return np.nan

    df_pivot['cobertura_agua'] = df_pivot.apply(calcula_percentual, axis=1)

    # Retornar apenas as colunas necessárias, já calculadas
    return df_pivot[['cod_municipio', 'ano', 'cobertura_agua']]

# ---
# FUNÇÃO PARA CALCULAR A COBERTURA DE ESGOTO
# ---
def calcular_cobertura_esgoto(df_raw: pd.DataFrame) -> pd.DataFrame:
    print("Calculando cobertura de esgoto...")

    # Converter para numérico
    df_raw['valor'] = pd.to_numeric(df_raw['valor'], errors='coerce')
    # CORREÇÃO DO FUTUREWARNING:
    df_raw['valor'] = df_raw['valor'].fillna(0) # Reatribuir em vez de usar inplace=True

    # ... (resto da função como antes) ...
    df_pivot = df_raw.pivot_table(
        index=['cod_municipio', 'ano'], columns='classificacao',
        values='valor', aggfunc='sum', fill_value=0
    ).reset_index()
    df_pivot.columns = df_pivot.columns.str.strip()
    def calcula_percentual(row): # ... (lógica como antes) ...
        total = row.get('Total', 0)
        if total == 0: return np.nan
        soma_rede_fossa = 0
        if row['ano'] == '2000':
            soma_rede_fossa = row.get('Rede geral de esgoto ou pluvial', 0) + row.get('Fossa séptica', 0)
        elif row['ano'] == '2010':
            soma_rede_fossa = (row.get('Tinham banheiro - de uso exclusivo do domicílio - rede geral de esgoto ou pluvial', 0) +
                               row.get('Tinham banheiro - de uso exclusivo do domicílio - fossa séptica', 0) +
                               row.get('Tinham sanitário - rede geral de esgoto ou pluvial', 0) +
                               row.get('Tinham sanitário - fossa séptica', 0))
        elif row['ano'] == '2022':
            soma_rede_fossa = (row.get('Rede geral, rede pluvial ou fossa ligada à rede', 0) +
                               row.get('Fossa séptica ou fossa filtro não ligada à rede', 0))
        return (soma_rede_fossa / total) * 100
    df_pivot['cobertura_esgoto'] = df_pivot.apply(calcula_percentual, axis=1)
    return df_pivot[['cod_municipio', 'ano', 'cobertura_esgoto']]

In [6]:
# ---
# INTERPOLAÇÃO
# ---
print("\nDados dos Censos processados. Iniciando interpolação...")

df_cobertura_agua = calcular_cobertura_agua(df_agua_raw)
df_cobertura_esgoto = calcular_cobertura_esgoto(df_esgoto_raw)

# Unir os dataframes de água e esgoto
df_saneamento_censos = pd.merge(df_cobertura_agua, df_cobertura_esgoto, on=['cod_municipio', 'ano'], how='outer')

# Criar o molde com todos os anos para a interpolação
anos_completos = range(2000, 2023)
municipios_unicos = df_saneamento_censos['cod_municipio'].unique()
idx = pd.MultiIndex.from_product([municipios_unicos, anos_completos], names=['cod_municipio', 'ano'])
df_interpolado = pd.DataFrame(index=idx).reset_index()

# Garantir que as chaves de merge tenham o mesmo tipo de dado (int64) em ambos os DataFrames
print("Garantindo consistência dos tipos de dados antes do merge...")
df_interpolado['cod_municipio'] = df_interpolado['cod_municipio'].astype('int64')
df_interpolado['ano'] = df_interpolado['ano'].astype('int64')
df_saneamento_censos['cod_municipio'] = df_saneamento_censos['cod_municipio'].astype('int64')
df_saneamento_censos['ano'] = df_saneamento_censos['ano'].astype('int64')
# ----------------------------------------------

# Juntar os dados dos censos no molde (agora com tipos de dados consistentes)
df_interpolado = pd.merge(df_interpolado, df_saneamento_censos, on=['cod_municipio', 'ano'], how='left')

# O resto do código de interpolação permanece o mesmo...
df_interpolado = df_interpolado.sort_values(by=['cod_municipio', 'ano'])
df_interpolado['cobertura_agua_linear'] = df_interpolado.groupby('cod_municipio')['cobertura_agua'].transform(lambda x: x.interpolate(method='linear'))
df_interpolado['cobertura_esgoto_linear'] = df_interpolado.groupby('cod_municipio')['cobertura_esgoto'].transform(lambda x: x.interpolate(method='linear'))

print("Aplicando interpolação polinomial (pchip) para uma curva mais suave...")

# Agrupar por município e aplicar a interpolação PCHIP
df_interpolado['cobertura_agua_suave'] = df_interpolado.groupby('cod_municipio')['cobertura_agua'].transform(
    # A mágica acontece aqui:
    lambda x: x.interpolate(method='pchip')
)
df_interpolado['cobertura_esgoto_suave'] = df_interpolado.groupby('cod_municipio')['cobertura_esgoto'].transform(
    lambda x: x.interpolate(method='pchip')
)

df_final = df_interpolado[df_interpolado['ano'].between(2002, 2021)].copy()
df_final[['cobertura_agua_linear', 'cobertura_esgoto_linear', 'cobertura_agua_suave', 'cobertura_esgoto_suave']] = df_final[['cobertura_agua_linear', 'cobertura_esgoto_linear', 'cobertura_agua_suave', 'cobertura_esgoto_suave']].round(2)
df_final = df_final.drop(columns=['cobertura_agua', 'cobertura_esgoto'])
print("\n--- Interpolação Concluída! ---")


Dados dos Censos processados. Iniciando interpolação...
Calculando cobertura de água com lógica unificada...
Calculando cobertura de esgoto...
Garantindo consistência dos tipos de dados antes do merge...
Aplicando interpolação polinomial (pchip) para uma curva mais suave...


/tmp/ipython-input-2641116840.py:13: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_raw['valor'].fillna(0, inplace=True)



--- Interpolação Concluída! ---


In [7]:
df_final.to_parquet('/content/drive/MyDrive/02 - Economia (UFPE)/2025.1/Tópicos de Macro (Economia do clima)/analise-saneamento-chuva-extrema-pe/build/output/cobertura_agua_esgoto_pe_2002_2021.parquet', index=False)